In [1]:
pwd

'/home/jupyter/Somatic_annotation'

# Quick sanity check for correct Parabricks installation and environment 


In [1]:
import subprocess

def check_cmd(cmd: str):
    print(f"$ {cmd}")
    result = subprocess.run(cmd, shell=True, capture_output=True, text=True)
    print(result.stdout)
    if result.returncode != 0:
        print(result.stderr)
        raise RuntimeError(f"Command failed with exit code {result.returncode}")

check_cmd("nvidia-smi")
check_cmd("pbrun --version")


$ nvidia-smi
Sun Nov 16 14:42:23 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.90.07              Driver Version: 550.90.07      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       On  |   00000000:00:04.0 Off |                    0 |
| N/A   40C    P8              9W /   70W |       1MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+----------------------------------

RuntimeError: Command failed with exit code 127

In [3]:
#Check for local docker image of the version installed 
!docker run --rm --gpus all nvcr.io/nvidia/clara/parabricks:v4.3.0-1 pbrun --version

Unable to find image 'nvcr.io/nvidia/clara/parabricks:v4.3.0-1' locally
docker: Error response from daemon: unauthorized: <html>
<head><title>401 Authorization Required</title></head>
<body>
<center><h1>401 Authorization Required</h1></center>
<hr><center>nginx/1.22.1</center>
</body>
</html>

Run 'docker run --help' for more information


In [1]:
pwd

'/home/jupyter/Parabricks_pathogen_Test'

In [2]:
!docker pull nvcr.io/nvidia/clara/clara-parabricks:4.6.0-1

4.6.0-1: Pulling from nvidia/clara/clara-parabricks

9dea01a6: Already exists 
09cfb5af: Already exists 
00b97743: Already exists 
7ecc44d6: Already exists 
15468fcc: Already exists 
987b81f9: Pulling fs layer 
8c224bf0: Pulling fs layer 
247cb961: Pulling fs layer 
105830d6: Pulling fs layer 
db69489a: Pulling fs layer 
5f3ce386: Pulling fs layer 
Digest: sha256:d0761eb4b9921bc046c53520287316d545eb79feaeb8f22387e9bb5734650447
Status: Downloaded newer image for nvcr.io/nvidia/clara/clara-parabricks:4.6.0-1
nvcr.io/nvidia/clara/clara-parabricks:4.6.0-1


In [5]:
!docker run --rm --gpus all nvcr.io/nvidia/clara/clara-parabricks:4.6.0-1 pbrun --version

Please visit https://docs.nvidia.com/clara/#parabricks for detailed documentation

pbrun: 4.6.0-1


In [7]:
pwd

'/home/jupyter/Parabricks_pathogen_Test'

# Download reference strain E. coli K-12 MG1655 genome and index it 

In [10]:
# Install NCBI datasets + unzip (skip if already installed)
!pip install --quiet ncbi-datasets-pylib
!sudo apt-get update -y && sudo apt-get install -y unzip


  DEPRECATION: Building 'ncbi-datasets-pylib' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'ncbi-datasets-pylib'. Discussion can be found at https://github.com/pypa/pip/issues/6334
  DEPRECATION: Building 'gffutils' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'gffutils'. Discussion can be found at https://github.com/pypa/pip/issues/6334
  You can safely remove it manually.
ERROR: pip's dependen

In [12]:
cd reference_mg1655

/home/jupyter/Parabricks_pathogen_Test/reference_mg1655


/opt/conda/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [13]:
pwd

'/home/jupyter/Parabricks_pathogen_Test/reference_mg1655'

# Running samtools inside PB container to creare .fai index file

In [16]:
import subprocess

REF_DIR = "/home/jupyter/Parabricks_pathogen_Test/reference_mg1655"
REF_FASTA = "Ecoli_K12_MG1655_ref.fasta"
IMG = "nvcr.io/nvidia/clara/clara-parabricks:4.6.0-1"

cmd = f"""
docker run --rm --gpus all \
    -v {REF_DIR}:/data \
    {IMG} \
    samtools faidx /data/{REF_FASTA}
"""

subprocess.run(cmd, shell=True, check=True)
print("Created FASTA index (.fai)")


Created FASTA index (.fai)


# GATK installation and run to create .dict file from ref fasta 

In [19]:
import os
import shutil
import subprocess

# ---- EDIT THESE ONLY IF YOUR PATHS ARE DIFFERENT ----
REF_DIR = "/home/jupyter/Parabricks_pathogen_Test/reference_mg1655"
REF_FASTA = "Ecoli_K12_MG1655_ref.fasta"
# -----------------------------------------------------

ref_path = os.path.join(REF_DIR, REF_FASTA)
dict_path = os.path.join(REF_DIR, "Ecoli_K12_MG1655_ref.dict")

if not os.path.exists(ref_path):
    raise FileNotFoundError(f"Reference FASTA not found: {ref_path}")

print(f"Reference FASTA: {ref_path}")
print(f"Dictionary will be written to: {dict_path}\n")

# 1) Check if gatk is already available
gatk_path = shutil.which("gatk")

if gatk_path:
    print(f"Found existing GATK at: {gatk_path}")
else:
    print("GATK not found in PATH. Installing gatk4 via conda...\n")
    # Try installing gatk4 with conda (requires conda in this environment)
    conda_path = shutil.which("conda")
    if not conda_path:
        raise RuntimeError(
            "conda is not available, cannot auto-install GATK. "
            "Please install GATK manually or add it to PATH."
        )
    # Install gatk4 into the current conda env
    install_cmd = "conda install -y -c bioconda gatk4"
    print(f"Running: {install_cmd}")
    subprocess.run(install_cmd, shell=True, check=True)
    
    # Re-check gatk in PATH
    gatk_path = shutil.which("gatk")
    if not gatk_path:
        raise RuntimeError(
            "GATK installation via conda completed but 'gatk' is still not in PATH. "
            "You may need to restart the kernel or ensure the conda env is activated."
        )
    print(f"\nGATK successfully installed at: {gatk_path}")

# 2) Run GATK CreateSequenceDictionary
print("\nRunning GATK CreateSequenceDictionary...")
cmd = (
    f"gatk CreateSequenceDictionary "
    f"-R {ref_path} "
    f"-O {dict_path}"
)
print(f"Command:\n{cmd}\n")

subprocess.run(cmd, shell=True, check=True)

# 3) Confirm output
if os.path.exists(dict_path):
    print("✅ Sequence dictionary created successfully:")
    print(dict_path)
    print("\nDirectory contents:")
    for f in os.listdir(REF_DIR):
        print("  ", f)
else:
    raise RuntimeError("GATK ran without error but the .dict file was not found.")


Reference FASTA: /home/jupyter/Parabricks_pathogen_Test/reference_mg1655/Ecoli_K12_MG1655_ref.fasta
Dictionary will be written to: /home/jupyter/Parabricks_pathogen_Test/reference_mg1655/Ecoli_K12_MG1655_ref.dict

GATK not found in PATH. Installing gatk4 via conda...

Running: conda install -y -c bioconda gatk4
Retrieving notices: done
Channels:
 - bioconda
 - conda-forge
Platform: linux-64
Solving environment: ...working... done




==> WARNING: A newer version of conda exists. <==
    current version: 25.5.1
    latest version: 25.9.1

Please update conda by running

    $ conda update -n base -c conda-forge conda





## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - gatk4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    alsa-lib-1.2.14            |       hb9d3cd8_0         553 KB  conda-forge
    ca-certificates-2025.11.12 |       hbd8a1cb_0         149 KB  conda-forge
    cairo-1.18.4               |       h3394656_0         955 KB  conda-forge
    certifi-2025.11.12         |     pyhd8ed1ab_0         153 KB  conda-forge
    conda-25.7.0               |  py310hff52083_0         949 KB  conda-forge
    font-ttf-dejavu-sans-mono-2.37|       hab24e00_0         388 KB  conda-forge
    font-ttf-inconsolata-3.000 |       h77eed37_0          94 KB  conda-forge
    font-ttf-source-code-pro-2.038|       h77eed37_0         684 KB  conda-forge
    font-ttf-ubuntu-0.83       |       h77eed37_3         1.5 MB  conda-forge
    fontconfig-2.15.0          |       h7e30c49_1   

Using GATK jar /opt/conda/share/gatk4-4.6.2.0-1/gatk-package-4.6.2.0-local.jar
Running:
    java -Dsamjdk.use_async_io_read_samtools=false -Dsamjdk.use_async_io_write_samtools=true -Dsamjdk.use_async_io_write_tribble=false -Dsamjdk.compression_level=2 -jar /opt/conda/share/gatk4-4.6.2.0-1/gatk-package-4.6.2.0-local.jar CreateSequenceDictionary -R /home/jupyter/Parabricks_pathogen_Test/reference_mg1655/Ecoli_K12_MG1655_ref.fasta -O /home/jupyter/Parabricks_pathogen_Test/reference_mg1655/Ecoli_K12_MG1655_ref.dict
18:12:37.348 INFO  NativeLibraryLoader - Loading libgkl_compression.so from jar:file:/opt/conda/share/gatk4-4.6.2.0-1/gatk-package-4.6.2.0-local.jar!/com/intel/gkl/native/libgkl_compression.so
[Sun Nov 16 18:12:37 UTC 2025] CreateSequenceDictionary --OUTPUT /home/jupyter/Parabricks_pathogen_Test/reference_mg1655/Ecoli_K12_MG1655_ref.dict --REFERENCE /home/jupyter/Parabricks_pathogen_Test/reference_mg1655/Ecoli_K12_MG1655_ref.fasta --TRUNCATE_NAMES_AT_WHITESPACE true --NUM_SEQUEN

Tool returned:
0
✅ Sequence dictionary created successfully:
/home/jupyter/Parabricks_pathogen_Test/reference_mg1655/Ecoli_K12_MG1655_ref.dict

Directory contents:
   Ecoli_K12_MG1655_ref.dict
   Ecoli_K12_MG1655_ref.fasta.fai
   Ecoli_K12_MG1655_ref.fasta


In [21]:
pwd

'/home/jupyter/Parabricks_pathogen_Test/reference_mg1655'

In [ ]:
import os
import shutil
import subprocess
import csv

# ---------------- CONFIG ----------------
PROJECT_ID = "PRJNA528413"
MAX_RUNS   = 5
BASE_DIR   = "/home/jupyter/Parabricks_pathogen_Test"
OUT_DIR    = os.path.join(BASE_DIR, "raw_reads_PRJNA528413")
RUNINFO_CSV = os.path.join(OUT_DIR, f"runinfo_{PROJECT_ID}.csv")
# ----------------------------------------

os.makedirs(OUT_DIR, exist_ok=True)

def run_cmd(cmd: str):
    """Run a shell command, raising on error and streaming output."""
    print(f"\n$ {cmd}")
    result = subprocess.run(cmd, shell=True)
    if result.returncode != 0:
        raise RuntimeError(f"Command failed with exit code {result.returncode}: {cmd}")

# 1) Ensure Entrez Direct (esearch, efetch) is available
if shutil.which("esearch") is None or shutil.which("efetch") is None:
    print("Entrez Direct not found. Installing via conda (bioconda/entrez-direct)...")
    if shutil.which("conda") is None:
        raise RuntimeError("conda is not available; cannot auto-install Entrez Direct. "
                           "Please install entrez-direct manually.")
    run_cmd("conda install -y -c bioconda entrez-direct")

# 2) Ensure SRA Toolkit (prefetch, fasterq-dump) is available
if shutil.which("prefetch") is None or shutil.which("fasterq-dump") is None:
    print("SRA Toolkit not found. Installing via conda (bioconda/sra-tools)...")
    if shutil.which("conda") is None:
        raise RuntimeError("conda is not available; cannot auto-install SRA Toolkit. "
                           "Please install sra-tools manually.")
    run_cmd("conda install -y -c bioconda sra-tools")

# 3) Use Entrez Direct to get RunInfo for this BioProject
print(f"\nFetching SRA RunInfo for BioProject {PROJECT_ID} ...")
# Note: using bash -lc so the pipe and redirect work correctly
run_cmd(
    f"bash -lc 'esearch -db sra -query \"{PROJECT_ID}[BioProject]\" | "
    f"efetch -format runinfo > \"{RUNINFO_CSV}\"'"
)

if not os.path.exists(RUNINFO_CSV):
    raise FileNotFoundError(f"RunInfo CSV not created: {RUNINFO_CSV}")

print(f"RunInfo saved to: {RUNINFO_CSV}")

# 4) Parse RunInfo and select up to MAX_RUNS run accessions
runs = []
with open(RUNINFO_CSV, newline="") as f:
    reader = csv.DictReader(f)
    for row in reader:
        run_id = row.get("Run")
        if run_id:
            runs.append(run_id)

if not runs:
    raise RuntimeError(f"No runs found in RunInfo for {PROJECT_ID}")

selected_runs = runs[:MAX_RUNS]
print(f"\nSelected {len(selected_runs)} runs from {PROJECT_ID}:")
for r in selected_runs:
    print("  ", r)

# 5) Download each run with prefetch and fasterq-dump
os.chdir(OUT_DIR)
print(f"\nDownloading FASTQ files into: {OUT_DIR}")

for run_id in selected_runs:
    print(f"\n=== Processing run {run_id} ===")

    # Download the .sra file into SRA cache
    run_cmd(f"prefetch {run_id}")

    # Convert SRA -> FASTQ (paired if applicable), write into OUT_DIR
    # -p = use multiple threads, -e 8 = 8 threads, -O . = output directory
    run_cmd(f"fasterq-dump -e 8 -p -O . {run_id}")

    # Compress outputs for storage efficiency
    # Handle both paired-end (_1/_2) and single-end (no suffix)
    fq1 = f"{run_id}_1.fastq"
    fq2 = f"{run_id}_2.fastq"
    fq_single = f"{run_id}.fastq"

    if os.path.exists(fq1):
        run_cmd(f"gzip -f {fq1}")
    if os.path.exists(fq2):
        run_cmd(f"gzip -f {fq2}")
    if os.path.exists(fq_single):
        run_cmd(f"gzip -f {fq_single}")

print("\n✅ All selected runs downloaded and compressed.")
print("Contents of download directory:")
for fname in sorted(os.listdir(OUT_DIR)):
    print("  ", fname)


Entrez Direct not found. Installing via conda (bioconda/entrez-direct)...

$ conda install -y -c bioconda entrez-direct
Channels:
 - bioconda
 - conda-forge
Platform: linux-64
Solving environment: ...working... done

## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - entrez-direct


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    entrez-direct-24.0         |       he881be0_0        16.3 MB  bioconda
    libidn2-2.3.8              |       hfac485b_1         136 KB  conda-forge
    libunistring-0.9.10        |       h7f98852_0         1.4 MB  conda-forge
    wget-1.21.4                |       hda4d442_0         752 KB  conda-forge
    ------------------------------------------------------------
                                           Total:        18.6 MB

The following NEW packages will be INSTALLED:

  entrez-direct      bioconda/linux-64::entrez-di



==> WARNING: A newer version of conda exists. <==
    current version: 25.7.0
    latest version: 25.9.1

Please update conda by running

    $ conda update -n base -c conda-forge conda





libunistring-0.9.10  | 1.4 MB    | 1          |   1% 


libidn2-2.3.8        | 136 KB    | #1         |  12% 

wget-1.21.4          | 752 KB    | 2          |   2% 


libidn2-2.3.8        | 136 KB    | ########## | 100% 


libidn2-2.3.8        | 136 KB    | ########## | 100% 

entrez-direct-24.0   | 16.3 MB   |            |   0% 
libunistring-0.9.10  | 1.4 MB    | ########## | 100% 

wget-1.21.4          | 752 KB    | ########## | 100% 

entrez-direct-24.0   | 16.3 MB   | #####7     |  58% 
libunistring-0.9.10  | 1.4 MB    | ########## | 100% 
                                                     
                                                     

                                                     


                                                      done
Preparing transaction: done
Verifying transaction: done
Executing transaction: done
SRA Toolkit not found. Installing via conda (bioconda/sra-tools)...

$ conda install -y -c bioconda sra-tools
Channels:
 - bioconda
 - conda-



==> WARNING: A newer version of conda exists. <==
    current version: 25.7.0
    latest version: 25.9.1

Please update conda by running

    $ conda update -n base -c conda-forge conda





## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - sra-tools


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    curl-8.13.0                |       h332b0f4_0         178 KB  conda-forge
    ncbi-vdb-3.2.1             |       h9948957_0        10.6 MB  bioconda
    ossuuid-1.6.2              |    h5888daf_1001          53 KB  conda-forge
    perl-5.32.1                | 7_hd590300_perl5        12.7 MB  conda-forge
    perl-alien-build-2.84      | pl5321h7b50bb2_1         195 KB  bioconda
    perl-alien-libxml2-0.17    | pl5321h577a1d6_1          14 KB  bioconda
    perl-business-isbn-3.007   | pl5321hdfd78af_0          18 KB  bioconda
    perl-business-isbn-data-20210112.006| pl5321hdfd78af_0          21 KB  bioconda
    perl-capture-tiny-0.48     | pl5321hdfd78af_2          19 KB  bioconda
    perl-carp-1.38             | pl5321hdfd78af_4          17 K

join   :|-------------------------------------------------- 100%
concat :|-------------------------------------------------- 100%
spots read      : 163,482
reads read      : 163,482
reads written   : 163,464
reads too short : 18



$ gzip -f SRR15415520.fastq


In [1]:
pwd

'/home/jupyter/Parabricks_pathogen_Test'

In [1]:
!pbrun minimap2

/bin/bash: line 1: pbrun: command not found


In [2]:
pwd

'/home/jupyter/Parabricks_pathogen_Test'

In [3]:
!pbrun --version

/bin/bash: line 1: pbrun: command not found


In [5]:
!find / -name pbrun 2>/dev/null

In [6]:
!docker run --rm --gpus all nvcr.io/nvidia/clara/clara-parabricks:4.6.0-1 pbrun --version

Please visit https://docs.nvidia.com/clara/#parabricks for detailed documentation

pbrun: 4.6.0-1


In [7]:
#Checking minimap2 test run with 1 sample 

In [8]:
!gunzip SRR15462566.fastq.gz

In [9]:
!docker run --rm --gpus all \
  -v $(pwd):/workdir \
  -w /workdir \
  nvcr.io/nvidia/clara/clara-parabricks:4.6.0-1 \
  pbrun minimap2 \
    --ref Ecoli_K12_MG1655_ref.fasta \
    --in-fq SRR15462566.fastq \
    --out-bam isolate_01_sorted.bam \
    --preset map-ont


[Parabricks Options Mesg]: Automatically generating ID prefix
[Parabricks Options Mesg]: Read group created for /workdir/SRR15462566.fastq
[Parabricks Options Mesg]: @RG\tID:SRR15462566.1\tLB:lib1\tPL:bar\tSM:sample\tPU:SRR15462566.1
[PB Info 2025-Nov-19 08:30:07] ------------------------------------------------------------------------------
[PB Info 2025-Nov-19 08:30:07] ||                 Parabricks accelerated Genomics Pipeline                 ||
[PB Info 2025-Nov-19 08:30:07] ||                              Version 4.6.0-1                             ||
[PB Info 2025-Nov-19 08:30:07] ||                                 minimap2                                 ||
[PB Info 2025-Nov-19 08:30:07] ------------------------------------------------------------------------------
[PB Info 2025-Nov-19 08:30:10] Reading reference file.
[PB Info 2025-Nov-19 08:30:10] -------------------------------------
[PB Info 2025-Nov-19 08:30:10] Elapsed-Minutes	Processed-Reads
[PB Info 2025-Nov-19 08:30:1

In [10]:
pwd

'/home/jupyter/Parabricks_pathogen_Test'

In [11]:
#Variant calling for haploid bacterial data with ont-germline and inbuilt DeepVariant tool for VCF conversion

In [12]:
!docker run --rm --gpus all \
  -v $(pwd):/workdir \
  -w /workdir \
  nvcr.io/nvidia/clara/clara-parabricks:4.6.0-1 \
  pbrun ont_germline \
    --ref Ecoli_K12_MG1655_ref.fasta \
    --in-bam isolate_01_sorted.bam \
    --out-variants isolate_01_variants.vcf

Please visit https://docs.nvidia.com/clara/#parabricks for detailed documentation


usage: pbrun ont_germline <options>
Help: pbrun ont_germline -h
pbrun: error: the following arguments are required: --out-bam


In [14]:
!docker run --rm --gpus all \
  -v $(pwd):/workdir \
  -w /workdir \
  nvcr.io/nvidia/clara/clara-parabricks:4.6.0-1 \
  pbrun ont_germline \
    --ref Ecoli_K12_MG1655_ref.fasta \
    --in-bam isolate_01_sorted.bam \
    --out-variants isolate_01_variants.vcf \
    --out-bam isolate_01_processed.bam

Please visit https://docs.nvidia.com/clara/#parabricks for detailed documentation


[Parabricks Options Mesg]: Read group created for /workdir/isolate_01_sorted.bam
[Parabricks Options Mesg]: @RG\tID:SRR15462566.1\tLB:lib1\tPL:bar\tSM:sample\tPU:SRR15462566.1
[Parabricks Options Mesg]: Setting --num-streams-per-gpu based on available device memory.
[Parabricks Options Mesg]: Please consider using --run-partition for best performance with more than 2 GPUs

[Parabricks Options Mesg]: Read group created for /workdir/isolate_01_sorted.bam
[Parabricks Options Mesg]: @RG\tID:SRR15462566.1\tLB:lib1\tPL:bar\tSM:sample\tPU:SRR15462566.1
[PB Info 2025-Nov-19 10:11:23] ------------------------------------------------------------------------------
[PB Info 2025-Nov-19 10:11:23] ||                 Parabricks accelerated Genomics Pipeline                 ||
[PB Info 2025-Nov-19 10:11:23] ||                              Version 4.6.0-1                             ||
[PB Info 2025-Nov-19 10:11:23] || 

In [3]:
#Startimg new 

In [4]:
!docker run --rm --gpus all \
  -v $(pwd):/workdir \
  -w /workdir \
  nvcr.io/nvidia/clara/clara-parabricks:4.6.0-1 \
  pbrun ont_germline \
    --ref Ecoli_K12_MG1655_ref.fasta \
    --in-bam isolate_01_sorted.bam \
    --out-variants isolate_01_variants.vcf \
    --out-bam isolate_01_processed.bam

Please visit https://docs.nvidia.com/clara/#parabricks for detailed documentation


[Parabricks Options Mesg]: Read group created for /workdir/isolate_01_sorted.bam
[Parabricks Options Mesg]: @RG\tID:SRR15462566.1\tLB:lib1\tPL:bar\tSM:sample\tPU:SRR15462566.1
[Parabricks Options Mesg]: Setting --num-streams-per-gpu based on available device memory.
[Parabricks Options Mesg]: Please consider using --run-partition for best performance with more than 2 GPUs

[Parabricks Options Mesg]: Read group created for /workdir/isolate_01_sorted.bam
[Parabricks Options Mesg]: @RG\tID:SRR15462566.1\tLB:lib1\tPL:bar\tSM:sample\tPU:SRR15462566.1
[PB Info 2025-Nov-19 12:24:19] ------------------------------------------------------------------------------
[PB Info 2025-Nov-19 12:24:19] ||                 Parabricks accelerated Genomics Pipeline                 ||
[PB Info 2025-Nov-19 12:24:19] ||                              Version 4.6.0-1                             ||
[PB Info 2025-Nov-19 12:24:19] || 

In [5]:
#Memory-intensive process, reduce reads per partition 

In [6]:
!docker run --rm --gpus all \
  --ipc=host \
  -v $(pwd):/workdir \
  -w /workdir \
  nvcr.io/nvidia/clara/clara-parabricks:4.6.0-1 \
  pbrun ont_germline \
    --ref Ecoli_K12_MG1655_ref.fasta \
    --in-bam isolate_01_sorted.bam \
    --out-variants isolate_01_variants.vcf \
    --out-bam isolate_01_processed.bam \
    --extra-flags "--max_reads_per_partition 50 --partition_size 10000"

Please visit https://docs.nvidia.com/clara/#parabricks for detailed documentation


usage: pbrun ont_germline <options>
Help: pbrun ont_germline -h
pbrun: error: unrecognized arguments: --extra-flags --max_reads_per_partition 50 --partition_size 10000


In [1]:
!docker run --rm --gpus all \
  -v $(pwd):/workdir \
  -w /workdir \
  nvcr.io/nvidia/clara/clara-parabricks:4.6.0-1 \
  pbrun ont_germline \
    --ref Ecoli_K12_MG1655_ref.fasta \
    --in-bam isolate_01_sorted.bam \
    --out-variants isolate_01_variants.vcf \
    --out-bam isolate_01_processed.bam

Please visit https://docs.nvidia.com/clara/#parabricks for detailed documentation


[Parabricks Options Mesg]: Read group created for /workdir/isolate_01_sorted.bam
[Parabricks Options Mesg]: @RG\tID:SRR15462566.1\tLB:lib1\tPL:bar\tSM:sample\tPU:SRR15462566.1
[Parabricks Options Mesg]: Setting --num-streams-per-gpu based on available device memory.
[Parabricks Options Mesg]: Please consider using --run-partition for best performance with more than 2 GPUs

[Parabricks Options Mesg]: Read group created for /workdir/isolate_01_sorted.bam
[Parabricks Options Mesg]: @RG\tID:SRR15462566.1\tLB:lib1\tPL:bar\tSM:sample\tPU:SRR15462566.1
[PB Info 2025-Nov-19 12:47:33] ------------------------------------------------------------------------------
[PB Info 2025-Nov-19 12:47:33] ||                 Parabricks accelerated Genomics Pipeline                 ||
[PB Info 2025-Nov-19 12:47:33] ||                              Version 4.6.0-1                             ||
[PB Info 2025-Nov-19 12:47:33] || 

In [2]:
!sudo journalctl -k --since "30 minutes ago" | grep "Killed process"

Nov 19 12:56:29 ac-genomics kernel: Out of memory: Killed process 29280 (deepsomatic) total-vm:531608348kB, anon-rss:421559076kB, file-rss:268736kB, shmem-rss:580288kB, UID:0 pgtables:893636kB oom_score_adj:0


In [3]:
!sudo journalctl -k --since "1 hour ago" | grep "Out of memory"

Nov 19 12:56:29 ac-genomics kernel: Out of memory: Killed process 29280 (deepsomatic) total-vm:531608348kB, anon-rss:421559076kB, file-rss:268736kB, shmem-rss:580288kB, UID:0 pgtables:893636kB oom_score_adj:0


In [4]:
!docker run --rm --gpus all \
  --ipc=host \
  -v $(pwd):/workdir \
  -w /workdir \
  nvcr.io/nvidia/clara/clara-parabricks:4.6.0-1 \
  pbrun ont_germline \
    --ref Ecoli_K12_MG1655_ref.fasta \
    --in-bam isolate_01_sorted.bam \
    --out-variants isolate_01_variants.vcf \
    --out-bam isolate_01_processed.bam \
    --low-memory \
    --run-partition


Please visit https://docs.nvidia.com/clara/#parabricks for detailed documentation


[Parabricks Options Mesg]: Read group created for /workdir/isolate_01_sorted.bam
[Parabricks Options Mesg]: @RG\tID:SRR15462566.1\tLB:lib1\tPL:bar\tSM:sample\tPU:SRR15462566.1
[Parabricks Options Mesg]: Setting --num-streams-per-gpu based on available device memory.

[Parabricks Options Mesg]: Read group created for /workdir/isolate_01_sorted.bam
[Parabricks Options Mesg]: @RG\tID:SRR15462566.1\tLB:lib1\tPL:bar\tSM:sample\tPU:SRR15462566.1
[PB Info 2025-Nov-19 13:13:30] ------------------------------------------------------------------------------
[PB Info 2025-Nov-19 13:13:30] ||                 Parabricks accelerated Genomics Pipeline                 ||
[PB Info 2025-Nov-19 13:13:30] ||                              Version 4.6.0-1                             ||
[PB Info 2025-Nov-19 13:13:30] ||                                 minimap2                                 ||
[PB Info 2025-Nov-19 13:13:30] -

In [7]:
!docker run --rm --gpus all \
  --memory 60g \
  --ipc=host \
  -e CUDA_VISIBLE_DEVICES="0,1,2,3" \
  -v $(pwd):/workdir \
  -w /workdir \
  nvcr.io/nvidia/clara/clara-parabricks:4.6.0-1 \
  pbrun ont_germline \
    --ref Ecoli_K12_MG1655_ref.fasta \
    --in-bam isolate_01_sorted.bam \
    --out-variants isolate_01_variants.vcf \
    --out-bam isolate_01_processed.bam \
    --low-memory \
    --run-partition

Please visit https://docs.nvidia.com/clara/#parabricks for detailed documentation


[Parabricks Options Mesg]: Read group created for /workdir/isolate_01_sorted.bam
[Parabricks Options Mesg]: @RG\tID:SRR15462566.1\tLB:lib1\tPL:bar\tSM:sample\tPU:SRR15462566.1
[Parabricks Options Mesg]: Setting --num-streams-per-gpu based on available device memory.

[Parabricks Options Mesg]: Read group created for /workdir/isolate_01_sorted.bam
[Parabricks Options Mesg]: @RG\tID:SRR15462566.1\tLB:lib1\tPL:bar\tSM:sample\tPU:SRR15462566.1
[PB Info 2025-Nov-19 13:25:20] ------------------------------------------------------------------------------
[PB Info 2025-Nov-19 13:25:20] ||                 Parabricks accelerated Genomics Pipeline                 ||
[PB Info 2025-Nov-19 13:25:20] ||                              Version 4.6.0-1                             ||
[PB Info 2025-Nov-19 13:25:20] ||                                 minimap2                                 ||
[PB Info 2025-Nov-19 13:25:20] -

In [1]:
docker run --rm \
  --gpus '"device=0,1,2,3"' \
  --memory 60g \
  --ipc=host \
  -v "$(pwd)":/workdir \
  -w /workdir \
  nvcr.io/nvidia/clara/clara-parabricks:4.6.0-1 \
  pbrun ont_germline \
    --ref Ecoli_K12_MG1655_ref.fasta \
    --in-bam isolate_01_sorted.bam \
    --out-variants isolate_01_variants.vcf \
    --out-bam isolate_01_processed.bam \
    --num-threads 32 \
    --num-streams-per-gpu 2 \
    --num-cpu-threads-per-stream 4 \
    --gpusort --gpuwrite \
    --low-memory \
    --run-partition \
    --gpu-num-per-partition 1


SyntaxError: invalid decimal literal (3026747042.py, line 3)

In [2]:
!docker run --rm --gpus all \
  --memory 60g \
  --ipc=host \
  -e CUDA_VISIBLE_DEVICES="0,1,2,3" \
  -v $(pwd):/workdir \
  -w /workdir \
  nvcr.io/nvidia/clara/clara-parabricks:4.6.0-1 \
  pbrun ont_germline \
    --ref Ecoli_K12_MG1655_ref.fasta \
    --in-bam isolate_01_sorted.bam \
    --out-variants isolate_01_variants.vcf \
    --out-bam isolate_01_processed.bam \
    --low-memory \
    --run-partition

Please visit https://docs.nvidia.com/clara/#parabricks for detailed documentation


[Parabricks Options Mesg]: Read group created for /workdir/isolate_01_sorted.bam
[Parabricks Options Mesg]: @RG\tID:SRR15462566.1\tLB:lib1\tPL:bar\tSM:sample\tPU:SRR15462566.1
[Parabricks Options Mesg]: Setting --num-streams-per-gpu based on available device memory.

[Parabricks Options Mesg]: Read group created for /workdir/isolate_01_sorted.bam
[Parabricks Options Mesg]: @RG\tID:SRR15462566.1\tLB:lib1\tPL:bar\tSM:sample\tPU:SRR15462566.1
[PB Info 2025-Nov-19 14:32:15] ------------------------------------------------------------------------------
[PB Info 2025-Nov-19 14:32:15] ||                 Parabricks accelerated Genomics Pipeline                 ||
[PB Info 2025-Nov-19 14:32:15] ||                              Version 4.6.0-1                             ||
[PB Info 2025-Nov-19 14:32:15] ||                                 minimap2                                 ||
[PB Info 2025-Nov-19 14:32:15] -

In [1]:
env | grep -iE "image|container"

env: |=grep -iE "image|container"


In [2]:
env | grep -iE "image|container"

env: |=grep -iE "image|container"


In [1]:
!pbrun --version

Please visit https://docs.nvidia.com/clara/#parabricks for detailed documentation

pbrun: 4.6.0-1


In [4]:
!pbrun ont-germlime -h

Please visit https://docs.nvidia.com/clara/#parabricks for detailed documentation

[Parabricks Options Error]: The command argument ont-germlime is not a valid Parabricks tools/pipeline
[Parabricks Options Error]: Run with -h to see help


In [5]:
!samtools --version

samtools 1.19.2
Using htslib 1.19.1
Copyright (C) 2024 Genome Research Ltd.

Samtools compilation details:
    Features:       build=configure curses=yes 
    CC:             gcc
    CPPFLAGS:       
    CFLAGS:         -Wall -g -O2
    LDFLAGS:        
    HTSDIR:         htslib-1.19.1
    LIBS:           
    CURSES_LIB:     -lncursesw

HTSlib compilation details:
    Features:       build=configure libcurl=yes S3=yes GCS=yes libdeflate=no lzma=yes bzip2=yes plugins=no htscodecs=1.6.0
    CC:             gcc
    CPPFLAGS:       
    CFLAGS:         -Wall -g -O2 -fvisibility=hidden
    LDFLAGS:        -fvisibility=hidden 

HTSlib URL scheme handlers present:
    built-in:	 preload, data, file
    S3 Multipart Upload:	 s3w, s3w+https, s3w+http
    Amazon S3:	 s3, s3+https, s3+http
    Google Cloud Storage:	 gs+http, gs, gs+https
    libcurl:	 ftp, http, https, ftps
    crypt4gh-needed:	 crypt4gh
    mem:	 mem


In [6]:
!gatk --version

Using GATK jar /usr/local/gatk/gatk-package-4.6.1.0-local.jar
Running:
    java -Dsamjdk.use_async_io_read_samtools=false -Dsamjdk.use_async_io_write_samtools=true -Dsamjdk.use_async_io_write_tribble=false -Dsamjdk.compression_level=2 -jar /usr/local/gatk/gatk-package-4.6.1.0-local.jar --version
The Genome Analysis Toolkit (GATK) v4.6.1.0
HTSJDK Version: 4.1.3
Picard Version: 3.3.0


In [7]:
!bcftools --version

bcftools 1.19
Using htslib 1.19
Copyright (C) 2023 Genome Research Ltd.
License Expat: The MIT/Expat license
This is free software: you are free to change and redistribute it.
There is NO WARRANTY, to the extent permitted by law.


In [8]:
!nvidia-smi

Wed Nov 19 21:51:14 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 580.82.07              Driver Version: 580.82.07      CUDA Version: 13.0     |
+-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          On  |   00000000:A0:1C.0 Off |                    0 |
| N/A   50C    P0             62W /  400W |       0MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [9]:
!pbrun ont_germline -h


Please visit https://docs.nvidia.com/clara/#parabricks for detailed documentation


usage: pbrun ont_germline <options>
Help: pbrun ont_germline -h

Run the germline pipeline from FASTQ/BAM to VCF by aligning long read ONT
sequences with minimap2 and using a deep neural network analysis.

options:
  -h, --help             show this help message and exit

Input Output file options:
  Options for Input and Output files for this tool.

  --ref REF              Path to the reference file.
  --index INDEX          Path to a minimizer index file generated by vanilla minimap2
                         to reduce indexing time.
  --in-fq IN_FQ          Path to a query sequence file in fastq or fastq.gz format.
  --in-bam IN_BAM        Path to the input BAM/CRAM file.
  --knownSites KNOWNSITES
                         Path to a known indels file. The file must be in vcf.gz
                         format. This option can be used multiple times.
  --interval-file INTERVAL_FILE
                    

In [1]:
!gunzip Parabricks_Pathogentest_Parabricks_pathogen_Test_isolate_01_sorted.bam.gz

In [2]:
!CUDA_VISIBLE_DEVICES="0,1" \
pbrun ont_germline \
  --ref Ecoli_K12_MG1655_ref.fasta \
  --in-bam isolate_01_sorted.bam \
  --out-variants isolate_01_variants.vcf \
  --out-bam isolate_01_processed.bam \
  --low-memory \
  --run-partition

Please visit https://docs.nvidia.com/clara/#parabricks for detailed documentation


[Parabricks Options Mesg]: Read group created for /opt/app-root/src/isolate_01_sorted.bam
[Parabricks Options Mesg]: @RG\tID:SRR15462566.1\tLB:lib1\tPL:bar\tSM:sample\tPU:SRR15462566.1
[Parabricks Options Mesg]: Setting --num-streams-per-gpu based on available device memory.


[Parabricks Options Mesg]: Read group created for /opt/app-root/src/isolate_01_sorted.bam
[Parabricks Options Mesg]: @RG\tID:SRR15462566.1\tLB:lib1\tPL:bar\tSM:sample\tPU:SRR15462566.1
/opt/parabricks/binaries/bin/minimap2: error while loading shared libraries: libtcmalloc_minimal.so.4: cannot open shared object file: No such file or directory
Process exited with error code: 127.
For technical support visit https://docs.nvidia.com/clara/index.html#parabricks
Exiting...

Could not run minimap2 as part of ont_germline pipeline
Exiting pbrun ...
